In [1]:
import json
import os

In [2]:
rebel_path = "/home/finapolat/GenIE/data/rebel"

test_data = []
with open(os.path.join(rebel_path, "en_test.jsonl"), 'r', encoding='utf-8') as f:
    for line in f:
        test_data.append(json.loads(line))
        
val_data = []    
with open(os.path.join(rebel_path, "en_val_small.jsonl"), 'r', encoding='utf-8') as f:
    for line in f:
        val_data.append(json.loads(line))

In [3]:
print(len(test_data))
print(len(val_data))

173601
15127


In [4]:
for instance in test_data[:5]:
    print(instance['input'])
    #print(instance['meta_obj'])
    print(instance['meta_obj']['substring_triples'])
    print(instance['meta_obj']['non_formatted_wikidata_id_output'])
    print() 

Coburg Peak (, ) is the rocky peak rising to 783 m in Erul Heights on Trinity Peninsula in Graham Land, Antarctica. 
[['Trinity Peninsula', 'part of', 'Graham Land'], ['Trinity Peninsula', 'continent', 'Antarctica'], ['Graham Land', 'continent', 'Antarctica']]
[['Q2038835', 'P361', 'Q618370'], ['Q2038835', 'P30', 'Q51'], ['Q618370', 'P30', 'Q51']]

Siniger Nunatak (, ‘Nunatak Siniger’ \'nu-na-tak si-'ni-ger\) is the rocky hill rising to over 500 m in the upper course of Russell East Glacier on Trinity Peninsula in Graham Land, Antarctica. 
[['Trinity Peninsula', 'part of', 'Graham Land'], ['Graham Land', 'continent', 'Antarctica']]
[['Q2038835', 'P361', 'Q618370'], ['Q618370', 'P30', 'Q51']]

Harald Kaas (19 May 1868 – 5 December 1953) was a Norwegian architect. 
[['Harald Kaas', 'date of birth', '19 May 1868'], ['Harald Kaas', 'date of death', '5 December 1953']]
[['Q5653840', 'P569', '1868-05-19T00:00:00Z^^http://www.w3.org/2001/XMLSchema#dateTime'], ['Q5653840', 'P570', '1953-12-05T

In [5]:
for instance in val_data[:5]:
    print(instance['input'])
    #print(instance['meta_obj'])
    print(instance['meta_obj']['substring_triples'])
    print(instance['meta_obj']['non_formatted_wikidata_id_output'])
    print() 

Chelopech Hill (, ‘Halm Chelopech’ \'h&lm che-lo-'pech\) is the ice-covered hill rising to 946 m in the north foothills of Detroit Plateau on Trinity Peninsula in Graham Land, Antarctica. 
[['Trinity Peninsula', 'part of', 'Graham Land'], ['Trinity Peninsula', 'continent', 'Antarctica'], ['Graham Land', 'continent', 'Antarctica']]
[['Q2038835', 'P361', 'Q618370'], ['Q2038835', 'P30', 'Q51'], ['Q618370', 'P30', 'Q51']]

Mount Gardner () is a mountain, high, standing west of Mount Tyree in the west-central part of the Sentinel Range, in the Ellsworth Mountains of Antarctica. 
[['Mount Gardner', 'mountain range', 'Sentinel Range'], ['Sentinel Range', 'mountain range', 'Ellsworth Mountains'], ['Ellsworth Mountains', 'has part', 'Sentinel Range']]
[['Q3321843', 'P4552', 'Q1778845'], ['Q1778845', 'P4552', 'Q1139110'], ['Q1139110', 'P527', 'Q1778845']]

Goodenough Glacier () is a broad sweeping glacier to the south of the Batterbee Mountains, flowing from the west shore of Palmer Land, Antarc

In [6]:
import torch
from transformers import pipeline

#generate_text = pipeline(model="databricks/dolly-v2-3b", torch_dtype=torch.bfloat16,
 #                        trust_remote_code=True, device_map="auto", return_full_text=True)
generate_text = pipeline(model="google/flan-t5-large", 
                         torch_dtype=torch.bfloat16, trust_remote_code=True,  
                         device_map="auto")#,
                         #return_full_text=True)

In [7]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline

# template for an instrution with no input
prompt = PromptTemplate(
    input_variables=["input_text"],
    template= """A triple has three components: subject, relations, object. Extract triples from the given text in the following format: (subject, relation, object). 
                Text to extract triples: {input_text} Extracted Triples: """)


In [8]:
hf_pipeline = HuggingFacePipeline(pipeline=generate_text)
llm_chain = LLMChain(llm=hf_pipeline, prompt=prompt, verbose=True)

In [9]:
for d in test_data[:5]:
    input_text = d["input"]
    prompt = prompt.format(input_text=input_text)
    #print(prompt)
    extraction = llm_chain.run(input_text=input_text)
    extraction2 = generate_text(prompt, max_length=1000, min_new_tokens=50, num_return_sequences=1, num_beams=1,)
    
    print(f"Input text: {input_text}")
    print(f"Extraction: {extraction}")
    print(f"Extraction2: {extraction2}")



> Entering new LLMChain chain...
Prompt after formatting:
A triple has three components: subject, relations, object. Extract triples from the given text in the following format: (subject, relation, object). 
                Text to extract triples: Coburg Peak (, ) is the rocky peak rising to 783 m in Erul Heights on Trinity Peninsula in Graham Land, Antarctica.  Extracted Triples: 

> Finished chain.
Input text: Coburg Peak (, ) is the rocky peak rising to 783 m in Erul Heights on Trinity Peninsula in Graham Land, Antarctica. 
Extraction: Coburg Peak, elevation, 783 m Coburg Peak, location, Erul Heights
Extraction2: [{'generated_text': 'Coburg Peak, elevation, 783 m Coburg Peak, location, Erul Heights Coburg Peak, location, Graham Land Graham Land, country, Antarctica Coburg Peak, elevation, 0°N, 0°E .'}]


> Entering new LLMChain chain...
Prompt after formatting:
A triple has three components: subject, relations, object. Extract triples from the given text in the following format: 

In [10]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

example_prompt = PromptTemplate(
    input_variables=["input_text", "target_triples"],
    template="""A triple has three components: subject, relations, object. Extract triples from the given text in the following format: (subject, relation, object). 
                Text to extract triples: {input_text} Extracted Triples: {target_triples}"""
)

In [11]:
examples = []

for instance in val_data[:5]:
    input_text = instance['input']
    target_triples = str(instance['meta_obj']['substring_triples'])
    examples.append({"input_text": input_text, "target_triples": target_triples})

In [12]:
for e in examples:
    print(e)    

{'input_text': "Chelopech Hill (, ‘Halm Chelopech’ \\'h&lm che-lo-'pech\\) is the ice-covered hill rising to 946 m in the north foothills of Detroit Plateau on Trinity Peninsula in Graham Land, Antarctica. ", 'target_triples': "[['Trinity Peninsula', 'part of', 'Graham Land'], ['Trinity Peninsula', 'continent', 'Antarctica'], ['Graham Land', 'continent', 'Antarctica']]"}
{'input_text': 'Mount Gardner () is a mountain, high, standing west of Mount Tyree in the west-central part of the Sentinel Range, in the Ellsworth Mountains of Antarctica. ', 'target_triples': "[['Mount Gardner', 'mountain range', 'Sentinel Range'], ['Sentinel Range', 'mountain range', 'Ellsworth Mountains'], ['Ellsworth Mountains', 'has part', 'Sentinel Range']]"}
{'input_text': 'Goodenough Glacier () is a broad sweeping glacier to the south of the Batterbee Mountains, flowing from the west shore of Palmer Land, Antarctica, into George VI Sound and the George VI Ice Shelf. ', 'target_triples': "[['Goodenough Glacier'

In [13]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # This is the list of examples available to select from.
    examples, 
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    #HuggingFaceInstructEmbeddings(), 
    HuggingFaceEmbeddings(),
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    Chroma, 
    # This is the number of examples to produce.
    k=2,
)

Using embedded DuckDB without persistence: data will be transient


In [14]:
example_formatter_template = """
Document to extract triples: <> {input_text} <>
Extracted triples from the give document: <> {target_triples} <>
"""
example_prompt = PromptTemplate(
    input_variables=["input_text", "target_triples"],
    template=example_formatter_template,
)

In [15]:
# Finally, we create the `FewShotPromptTemplate` object.
few_shot_prompt = FewShotPromptTemplate(
    # These are the examples we want to insert into the prompt.
    example_selector=example_selector,
    # This is how we want to format the examples when we insert them into the prompt.
    example_prompt=example_prompt,
    # The prefix is some text that goes before the examples in the prompt.
    # Usually, this consists of intructions.
    prefix="""A triple contains three componants: subject, relation, object. Extract extensive set of triples from the given document. 
    Here are some examples: """,
    # The suffix is some text that goes after the examples in the prompt.
    # Usually, this is where the user input will go
    suffix="""End of the examples.
    Document to extract triples: <> {input} <> \n Extracted triples from the given document: <> """,
    # The input variables are the variables that the overall prompt expects.
    input_variables=["input"],
    # The example_separator is the string we will use to join the prefix, examples, and suffix together with.
    example_separator="\n",
)

In [16]:
print(few_shot_prompt.format(input="Steve Bright is the creator of Bananaman which was last broadcast on April 15th 1986 by the BBC."))

A triple contains three componants: subject, relation, object. Extract extensive set of triples from the given document. 
    Here are some examples: 

Document to extract triples: <> Mount Gardner () is a mountain, high, standing west of Mount Tyree in the west-central part of the Sentinel Range, in the Ellsworth Mountains of Antarctica.  <>
Extracted triples from the give document: <> [['Mount Gardner', 'mountain range', 'Sentinel Range'], ['Sentinel Range', 'mountain range', 'Ellsworth Mountains'], ['Ellsworth Mountains', 'has part', 'Sentinel Range']] <>


Document to extract triples: <> Goodenough Glacier () is a broad sweeping glacier to the south of the Batterbee Mountains, flowing from the west shore of Palmer Land, Antarctica, into George VI Sound and the George VI Ice Shelf.  <>
Extracted triples from the give document: <> [['Goodenough Glacier', 'instance of', 'glacier'], ['Palmer Land', 'continent', 'Antarctica']] <>

End of the examples.
    Document to extract triples: <>

In [17]:
llm_fewshot_chain = LLMChain(llm=hf_pipeline, prompt=few_shot_prompt, verbose=True)

In [18]:
for d in test_data[:5]:
    input_text = d["input"]
    extraction = llm_fewshot_chain.run(input=input_text)
    print(f"Input text: {input_text}")
    print(f"Extraction: {extraction}")



> Entering new LLMChain chain...
Prompt after formatting:
A triple contains three componants: subject, relation, object. Extract extensive set of triples from the given document. 
    Here are some examples: 

Document to extract triples: <> Chelopech Hill (, ‘Halm Chelopech’ \'h&lm che-lo-'pech\) is the ice-covered hill rising to 946 m in the north foothills of Detroit Plateau on Trinity Peninsula in Graham Land, Antarctica.  <>
Extracted triples from the give document: <> [['Trinity Peninsula', 'part of', 'Graham Land'], ['Trinity Peninsula', 'continent', 'Antarctica'], ['Graham Land', 'continent', 'Antarctica']] <>


Document to extract triples: <> Mount Gardner () is a mountain, high, standing west of Mount Tyree in the west-central part of the Sentinel Range, in the Ellsworth Mountains of Antarctica.  <>
Extracted triples from the give document: <> [['Mount Gardner', 'mountain range', 'Sentinel Range'], ['Sentinel Range', 'mountain range', 'Ellsworth Mountains'], ['Ellsworth Mou

/home/finapolat/.local/lib/python3.10/site-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(



> Finished chain.
Input text: Coburg Peak (, ) is the rocky peak rising to 783 m in Erul Heights on Trinity Peninsula in Graham Land, Antarctica. 
Extraction: [['Coburg Peak', 'POSITIONS', 'Erul


> Entering new LLMChain chain...
Prompt after formatting:
A triple contains three componants: subject, relation, object. Extract extensive set of triples from the given document. 
    Here are some examples: 

Document to extract triples: <> Chelopech Hill (, ‘Halm Chelopech’ \'h&lm che-lo-'pech\) is the ice-covered hill rising to 946 m in the north foothills of Detroit Plateau on Trinity Peninsula in Graham Land, Antarctica.  <>
Extracted triples from the give document: <> [['Trinity Peninsula', 'part of', 'Graham Land'], ['Trinity Peninsula', 'continent', 'Antarctica'], ['Graham Land', 'continent', 'Antarctica']] <>


Document to extract triples: <> Rutgers Glacier () is a steep glacier on the west side of the Royal Society Range in the Ross Dependency, Antarctica.  <>
Extracted triples fr